In [ ]:
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
xls = pd.ExcelFile('/content/drive/MyDrive/Project/Paper1/Data/Clinical Data/Original.xlsx')
Sheet_4 = pd.read_excel(xls, 'Visite')
Sheet_1 = pd.read_excel(xls, 'Anagrafica')
data1 = Sheet_4.copy()
data2 = Sheet_1.copy()
# In order to optain statis features, We need to merge to sheet together
final_data = pd.merge(data1,data2,how = 'left', left_on=['Paziente ID'], right_on = ['Paziente ID'])
final_data.columns

In [ ]:
#Select important feature to impute
final_data = final_data[["Id", "Paziente ID",'Data visita','Piramidale','Cerebellare','Troncoencefalica','Sensitiva','Sfinteriche',
                         'Visiva','Mentali','Deambulazione','Punteggio EDSS valutato dal clinico','Data di nascita',
                         'Data di morte','Sesso','SM in età pedriatrica']]

# Translate the header
final_data.columns = ['Id', 'Patient_ID', 'Date_of_visit', 'Pyramidal', 'Cerebellar',
       'Thronchioencephalic', 'Sensitive', 'Sphincteric', 'Visual', 'Mental',
       'Deambulation','EDSS_score_assessed_by_clinician', 'Date of birth',
       'Data of death','Sex', 'MS in pediatric age']


# Remove patient without target value (Y) and converting object feature to datetime
final_data = final_data[final_data['EDSS_score_assessed_by_clinician'].notna()]
final_data['Date_of_visit'] = pd.to_datetime(final_data['Date_of_visit'])
final_data['Date of birth'] = pd.to_datetime(final_data['Date of birth'])


le = preprocessing.LabelEncoder() # Use LabelEncoder to assign a label to each string Sex and MS in pediatric age feature.
final_data['Sex'] = le.fit_transform(final_data['Sex']) # 0 = Female, 1 = Male
final_data['MS in pediatric age'] = le.fit_transform(final_data['MS in pediatric age']) # 0 = NO, 1 = Yes

(final_data.isnull().sum()/ len(final_data))*100 # Missingness rate for each feature

patient_number = len(np.unique(final_data['Patient_ID'])) # Getting the number of patient
followup_count_patient = final_data.groupby('Patient_ID')['Id'].count().reset_index() #Getting the number of followup data for each patient

In [ ]:
# Delete patients with only one onservation which is completely missing
final_data_1 = final_data.copy() 
for i in np.unique(final_data_1["Patient_ID"]):
    patient_data = final_data_1[final_data_1["Patient_ID"]==i]
    missing_rate = (patient_data.isnull().sum(axis=1)/patient_data.shape[1])*100
    if len(patient_data)==1 and missing_rate.values[0]>=50:
    final_data_1.drop(patient_data.index, inplace=True)

In [ ]:
final_data_1['Date_of_visit'] = pd.to_datetime(final_data_1['Date_of_visit'])
final_data_1['Date of birth'] = pd.to_datetime(final_data_1['Date of birth'])
visit_time = np.array(final_data_1['Date_of_visit'])
birth_time = np.array(final_data_1['Date of birth'])
visit_year = np.array(visit_time, dtype='datetime64[Y]')
birth_year = np.array(birth_time, dtype='datetime64[Y]')
visit_month = np.array(visit_time, dtype='datetime64[M]')
birth_month = np.array(birth_time, dtype='datetime64[M]')
visit_day = np.array(visit_time, dtype='datetime64[D]')
birth_day = np.array(birth_time, dtype='datetime64[D]')
age = visit_year - birth_year - ((visit_month < birth_month) | ((visit_month == birth_month) & (visit_day < birth_day)))
final_data_1.to_csv("Updated_MS.csv", index= False)